In [ ]:
import pandas as pd
import numpy as np
import pulp

# The Assignment Problem

The Assignment Problem is a special case of the Transportation Problem, where all the supplies and all the demands are either zero or one.

<img src="The Assignment Problem image 01.png">

In this scenario, the hospital Sørlandet Sykehus has received a number of calls from people who are in need of assistance. Luckily, the hospital have more than enough ambulances, so the problem is only a matter of figuring out which ambulance to send where. The ambulances are all located at different locations, so the distance between each ambulance and each patient is different. Based on the GPS-position of the ambulances and the patients, we get an estimate of the time it would take any ambulance to drive to any patient in the following table:

## Prepare the data

In [ ]:
Ambulances = ['Ambulance 1', 'Ambulance 2', 'Ambulance 3', 'Ambulance 4', 'Ambulance 5']
Patients = ['Patient 1', 'Patient 2', 'Patient 3', 'Patient 4', 'Patient 5']
data = [
    [20, 14,  6, 10, 22],
    [16,  8, 22, 20, 10],
    [ 8,  6, 24, 14, 12],
    [20, 22,  2,  8,  6],
    [ 4, 16, 22,  6, 24]
]

df = pd.DataFrame(data=data, index=Ambulances, columns=Patients)
df

## Create the variables

In the assignment problem, we generate a variable for each possible scenario. The variable $x_{a,p}$ will be binary, and corresponds to whether or not ambulance $a$ drives to patient $p$.

<img src="The Assignment Problem image 02.png">

In [ ]:
x = pulp.LpVariable.dicts("scenario",
                         ((a, p) for a in Ambulances for p in Patients),
                         cat='Binary')
x

## Initiate an empty LP Problem

In [ ]:
prob = pulp.LpProblem("TransportationProblem", pulp.LpMinimize)

## Create the constraints

For this formulation, it is important that each car can only choose on scenario, meaning it should visit one patient, and one patient only.

### One patient per ambulance
Since the variables for any ambulance is binary, we can make a constraint that says that the sum of the variables from any ambulance must equal one. This way we force each ambulance to only choose one patient to visit.

In [ ]:
for a in Ambulances:
    prob += pulp.lpSum([x[a,p] for p in Patients]) == 1, f"{a} can only visit one patient"

### One ambulance per patient

The constraint above ensures that one ambulance only visits one patient. However, is does not enforce that all patients must be visited. Neither does it enforce that a patient should only be visited by one ambulance. We therefor need a constraint that ensures that each patient can only be visited by one ambulance.

In [ ]:
for p in Patients:
    prob += pulp.lpSum([x[a,p] for a in Ambulances]) == 1, f"{p} can only be visited by one ambulance"

## Create the objective function
If any scenario $x_{a,p}$ is carried out, it will bring with it a cost in the form of the time it will take the ambulance to visit the corresponding patient. The objective function is therefor to minimize the sum of each scenario variable multiplied by the time it would take to coduct that scenario

In [ ]:
prob += pulp.lpSum([df.loc[a,p] * x[a,p] for a in Ambulances for p in Patients])

## Find the optimal solution

In [ ]:
prob.solve()
status = pulp.LpStatus[prob.status]
obj_value = prob.objective.value()

print(f"The solver found a solution that is *{status}*, where the total time spent driving to patients is {obj_value:.1f} minutes")

In [ ]:
results = pd.DataFrame(data=[[x[a,p].value()for p in Patients] for a in Ambulances], index=Ambulances, columns=Patients, dtype=bool)
results

#  Final notes:
In this example, the number of ambulances matched the number of patients. This might not be the case in a real life scenario, where two other cases might occure:
1. If you have more ambulances than pasients, change the $=$-constraint to a $\leq$-constraint in the section where we enforced that _"any ambulance {$a$} can only visit one patient"_. This will enable some ambulances to visit zero patients.
2. If you have more pasients than ambulances, change the $=$-constraint to a $\leq$-constraint in the section where we enforced that _"any patient {$p$} can only be visited by one ambulance"_. This is a grimmer solution, though, as it allows the ambulances to not visit the patients that would take too long to visit. Nonetheless, in situations where there are more patients than the doctors are able to treat, these kinds of decisions sometimes have to be made. However, for further work on this case, a better solution might be to factor in severity in the objective function. This way, the optimization algorithm would consider e.g. a weighted sum between the "chance of survival" and the time it would take an ambulance to get there.